In [1]:
import numpy as np
from gtsam import *	
from gpmp2 import *
from gpmp_utils.generate2Ddataset import generate2Ddataset
from gpmp_utils.signedDistanceField2D import signedDistanceField2D
from gpmp_utils.generateArm import generateArm


In [2]:
dataset = generate2Ddataset('OneObstacleDataset')

In [3]:
rows = dataset.rows
cols = dataset.cols
cell_size = dataset.cell_size
origin_point2 = Point2(dataset.origin_x, dataset.origin_y)

In [4]:
# signed distance field
field = signedDistanceField2D(dataset.map, cell_size)
sdf = PlanarSDF(origin_point2, cell_size, field)


In [5]:
# settings
total_time_sec = 5.0
total_time_step = 10
total_check_step = 50
delta_t = total_time_sec / total_time_step
check_inter = total_check_step / total_time_step - 1

# use GP interpolation
use_GP_inter = True


In [6]:
# arm model
arm = generateArm('SimpleTwoLinksArm')

In [7]:
# GP
Qc = np.identity(2)
Qc_model = noiseModel_Gaussian.Covariance(Qc)

# Obstacle avoid settings
cost_sigma = 0.1
epsilon_dist = 0.1

In [8]:
# prior to start/goal
pose_fix = noiseModel_Isotropic.Sigma(2, 0.0001)
vel_fix = noiseModel_Isotropic.Sigma(2, 0.0001)

In [12]:
# start and end conf
start_conf = np.asarray([0, 0])
start_vel = np.asarray([0, 0])
end_conf = np.asarray([np.pi/2, 0])
end_vel = np.asarray([0, 0])
avg_vel = (end_conf / total_time_step) / delta_t

In [13]:
#%% init optimization
graph = NonlinearFactorGraph()
init_values = Values()

In [14]:
for i in range(0, total_time_step+1):
    key_pos = symbol(ord('x'), i) #TODO: check this mustafa 
    key_vel = symbol(ord('v'), i)
    
    #% initialize as straight line in conf space
    pose = start_conf * (total_time_step-i)/total_time_step + end_conf * i/total_time_step
    vel = avg_vel
    init_values.insert(key_pos, pose)
    init_values.insert(key_vel, vel)
    
    #% start/end priors
    if i==0:
        graph.add(PriorFactorVector(key_pos, start_conf, pose_fix))
        graph.add(PriorFactorVector(key_vel, start_vel, vel_fix))
    elif i==total_time_step:
        graph.add(PriorFactorVector(key_pos, end_conf, pose_fix))
        graph.add(PriorFactorVector(key_vel, end_vel, vel_fix))
    
    #% GP priors and cost factor
    if i > 0:
        key_pos1 = symbol(ord('x'), i-1)
        key_pos2 = symbol(ord('x'), i)
        key_vel1 = symbol(ord('v'), i-1)
        key_vel2 = symbol(ord('v'), i)

        graph.add(GaussianProcessPriorLinear(key_pos1, key_vel1,
            key_pos2, key_vel2, delta_t, Qc_model))
        
        #% cost factor
        graph.add(ObstaclePlanarSDFFactorArm(key_pos, arm, 
                                sdf, cost_sigma, epsilon_dist))
        
        #% GP cost factor
        if use_GP_inter and check_inter > 0:
            for j in range(1, check_inter+1):
                tau = j * (total_time_sec / total_check_step)
                graph.add(ObstaclePlanarSDFFactorGPArm(
                    key_pos1, key_vel1, key_pos2, key_vel2,
                    arm, sdf, cost_sigma, epsilon_dist,
                    Qc_model, delta_t, tau))

In [15]:
#%% optimize!
use_trustregion_opt = False

if use_trustregion_opt:
    parameters = DoglegParams()
    parameters.setVerbosity('ERROR')
    optimizer = DoglegOptimizer(graph, init_values, parameters)
else:
    parameters = GaussNewtonParams()
    parameters.setVerbosity('ERROR')
    optimizer = GaussNewtonOptimizer(graph, init_values, parameters)

In [16]:
optimizer.optimize()
result = optimizer.values()

In [17]:
print(result)

Values with 22 values:
Value v0: (N5Eigen6MatrixIdLin1ELi1ELi0ELin1ELi1EEE) [
 	0;
    	0
  ]

Value v1: (N5Eigen6MatrixIdLin1ELi1ELi0ELin1ELi1EEE) [
 	0.314159;
    	       0
  ]

Value v2: (N5Eigen6MatrixIdLin1ELi1ELi0ELin1ELi1EEE) [
 	0.314159;
    	       0
  ]

Value v3: (N5Eigen6MatrixIdLin1ELi1ELi0ELin1ELi1EEE) [
 	0.314159;
    	       0
  ]

Value v4: (N5Eigen6MatrixIdLin1ELi1ELi0ELin1ELi1EEE) [
 	0.314159;
    	       0
  ]

Value v5: (N5Eigen6MatrixIdLin1ELi1ELi0ELin1ELi1EEE) [
 	0.314159;
    	       0
  ]

Value v6: (N5Eigen6MatrixIdLin1ELi1ELi0ELin1ELi1EEE) [
 	0.314159;
    	       0
  ]

Value v7: (N5Eigen6MatrixIdLin1ELi1ELi0ELin1ELi1EEE) [
 	0.314159;
    	       0
  ]

Value v8: (N5Eigen6MatrixIdLin1ELi1ELi0ELin1ELi1EEE) [
 	0.314159;
    	       0
  ]

Value v9: (N5Eigen6MatrixIdLin1ELi1ELi0ELin1ELi1EEE) [
 	0.314159;
    	       0
  ]

Value v10: (N5Eigen6MatrixIdLin1ELi1ELi0ELin1ELi1EEE) [
 	0;
    	0
  ]

Value x0: (N5Eigen6MatrixIdLin1ELi1ELi0ELin1ELi1EEE) [
 	0

In [20]:
conf = result.atVector(symbol(ord('x'), 10))

In [21]:
print(conf)

[1.57079633 0.        ]
